<a href="https://colab.research.google.com/github/cl-liamchiu/caption-generator/blob/main/%E5%AD%97%E5%B9%95%E6%AA%94%E6%A1%88%E7%94%A2%E7%94%9F%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

#@markdown #使用教學

#@markdown 請從上方選單中選擇【檔案】→【在雲端硬碟中儲存複本】，然後在你的複本中執行下列動作(可以關掉原來的)。

#@markdown 在你的google雲端中建立一個資料夾，資料夾內放入你要產生字幕的影片(可以多部)，並在下面填寫資料夾名稱。

#@markdown 假設你的影片放在google雲端中名為「影片」的資料夾中，請在下方填入「影片」。如果你的影片放在「影片」資料夾裡面的「20230820」資料夾中，請填入「影片/20230820」，以此類推。
from google.colab import drive
drive.mount('/content/drive')
folder_name = "影片" #@param {type:"string"}

! pip install stable-ts


import stable_whisper
import os
from moviepy.video.io.VideoFileClip import VideoFileClip

model_name = "large-v3"

model = stable_whisper.load_model(model_name)
model.device

#@markdown 如果要在字幕檔案產生後，要刪除原本影片的話，請勾選下列空格，預設是不刪除。
delete_original_video = False #@param {type:"boolean"}

#@markdown 設定好後點案程式碼左上角的開始鍵來執行程式。一開始會詢問是否可以連結到你的雲端硬碟，請允許它連結到你的雲端硬碟。之後就讓程式跑，跑完之後，在原本影片的資料夾下就會出現.srt檔案，那個就是字幕檔，將它下載並導入到你的影片編輯軟體中。

#@markdown 字幕產生時間，1分鐘影片慢的話大概30秒，10分鐘影片慢的話大概5分鐘。

def extract_audio(video_path):
    audio_path = video_path[:-4]+".wav"

    with VideoFileClip(video_path) as video_clip:
        audio_clip = video_clip.audio
        audio_clip.write_audiofile(audio_path, codec='pcm_s16le')

    if delete_original_video == True:
      try:
          os.remove(video_path)
          # print(f"File '{video_path}' deleted successfully.")
      except OSError as e:
          print(f"Error deleting file '{video_path}': {e}")

    return audio_path


def audio_to_text(audio_path):
    srt_path = audio_path[:-4]+".srt"
    result = model.transcribe(audio_path, language='zh',
                              initial_prompt="以下是普通話的句子",
                              )

    result.to_srt_vtt(srt_path, word_level=False)

    try:
        os.remove(audio_path)
        # print(f"File '{audio_path}' deleted successfully.")
    except OSError as e:
        print(f"Error deleting file '{audio_path}': {e}")

    return srt_path

# 設定資料夾路徑
folder_path = "/content/drive/MyDrive/" + folder_name

# 使用os.listdir()函式來取得資料夾內的檔案列表
file_list = os.listdir(folder_path)

# 產生字幕檔
for video_name in file_list:
  try:
    video_path = os.path.join(folder_path, video_name)
    audio_to_text(extract_audio(video_path))
  except:
    print(video_name)